In [1]:
import pandas as pd
import nltk
from nltk import ngrams
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#import pyLDAvis.gensim
import gensim
import pickle 
import pyLDAvis
pyLDAvis.enable_notebook()
import pyLDAvis.gensim
import os
import gensim.corpora as corpora
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/JaredJose/Roblox-Scraper/main/data/ALLRoblox-securityprivacy.csv?token=GHSAT0AAAAAAB6VRXEY6X6YMDEWPQFHU7AUZALPAIA')

In [3]:
df.head()

,type,url,title,author,date,timestamp,score,text,comment_id,parent_id
0,Post,https://www.reddit.com/gallery/10z3bmp,I spent the whole day talking to people. Close...,dextergoon,2023-02-10,1.676064e+09,4,I spent the whole day talking to people. Close...,NaN,NaN
1,comment,https://www.reddit.com/gallery/10z3bmp,NaN,biolatic,2023-02-10,1.676067e+09,1,"You should contact Roblox support, see what th...",j81bh4e,NaN
2,comment,https://www.reddit.com/gallery/10z3bmp,NaN,ML4Bzero,2023-02-11,1.676116e+09,1,"You're probably temporarily suspended, happene...",j83mq5i,NaN
3,comment,https://www.reddit.com/gallery/10z3bmp,NaN,MammothSouthern7717,2023-02-25,1.677360e+09,1,Deserve it if you think giving your id to robl...,ja05q7i,NaN
4,comment,https://www.reddit.com/gallery/10z3bmp,NaN,cool91725,2023-02-11,1.676081e+09,1,Probably nothing. Roblox support sucks,j828nd8,j81bh4e


In [4]:
df['text']

0       I spent the whole day talking to people. Close...
1       You should contact Roblox support, see what th...
2       You're probably temporarily suspended, happene...
3       Deserve it if you think giving your id to robl...
4                  Probably nothing. Roblox support sucks
                              ...                        
6393    Generally, it is not that obvious, most of the...
6394    What’s up with the voice chat update\nI’ve had...
6395    No VC is still there, pretty sure you need to ...
6396    You also have to go the privacy i think and en...
6397                                After you're verified
Name: text, Length: 6398, dtype: object

In [5]:
#df['comment'].str.count('game').sum()

In [6]:
text = " ".join(df['text'].dropna().astype(str))
tokens = nltk.word_tokenize(text)
tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]
stop_words = set(stopwords.words('english'))
filtered_tokens = [token for token in tokens if token.lower() not in stop_words and token.strip() != '']


bi_grams = ngrams(filtered_tokens, 2)

In [7]:
bi_grams_list = list(bi_grams)
bi_grams_list = [tuple(filter(None, bg)) for bg in bi_grams_list]
#print(bi_grams_list)

In [8]:
bi_grams_list1 = [' '.join(bi_gram) for bi_gram in bi_grams_list if '(' not in bi_gram and ')' not in bi_gram]

freq_dist = nltk.FreqDist(bi_grams_list1)
most_common = freq_dist.most_common(1)
print('The most common bi-gram is:', most_common[0][0], 'with a frequency of', most_common[0][1])

The most common bi-gram is: ca nt with a frequency of 222


In [9]:
vectorizer = CountVectorizer(ngram_range = (2, 2))
X1=vectorizer.fit_transform(bi_grams_list1)
features = (vectorizer.get_feature_names_out())

vectorizer = TfidfVectorizer(ngram_range = (2, 2))
tfidf = vectorizer.fit_transform(bi_grams_list1)


sums = tfidf.sum(axis = 0)
data1 = []
for col, term in enumerate(features):
    data1.append( (term, sums[0, col] ))
ranking = pd.DataFrame(data1, columns = ['term', 'rank'])
words = (ranking.sort_values('rank', ascending = False))
print ("\n\nWords : \n", words.head(7))



Words : 
                  term   rank
9615            ca nt  233.0
54085  roblox support   97.0
42751         nt know   89.0
71671           wo nt   77.0
37805       make sure   75.0
73242        year old   70.0
26688          gon na   66.0


In [10]:
df_ques = df['text'].str.endswith('?')
df_true = df.loc[df_ques]
print(df_true['text'])

5       Parents, watch your damn kids.\nMy 7 y/o daugh...
37      There are parental controls you can use, as we...
63                 why are all the parts named "zarm"????
94                 Is it the same for the roblox website?
105                         How did you get the giftcard?
                              ...                        
6370    Voice chat for under 13\nIs it possible to act...
6376              they never said they were under 13 tho?
6378               Pardon, what does the word "imo" mean?
6382    For parents (or those that care about kids saf...
6394    What’s up with the voice chat update\nI’ve had...
Name: text, Length: 668, dtype: object


In [11]:
df['text_processed'] = df['text'].apply(lambda x: re.sub('[,.!?]', '', x) if isinstance(x, str) else x)
df['text_processed'] = df['text_processed'].apply(lambda x: x.lower() if isinstance(x, str) else x)

df['text_processed'].head()


0    i spent the whole day talking to people closer...
1    you should contact roblox support see what the...
2    you're probably temporarily suspended happened...
3    deserve it if you think giving your id to robl...
4                probably nothing roblox support sucks
Name: text_processed, dtype: object

In [12]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = df.text.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['spent', 'whole', 'day', 'talking', 'people', 'closer', 'night', 'took', 'away', 'voice', 'chat', 'without', 'explanation']


In [13]:
id2word = corpora.Dictionary(data_words)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]
num_topics = 10
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
#s.makedirs('./results/ldavis_prepared_10')
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared
pyLDAvis.display(LDAvis_prepared)

/home/zhz094/.local/lib/python3.9/site-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
